In [40]:
import anthropic
import json

class Tool:
    def __init__(self, schema, callable):
        self.schema = schema
        self.callable = callable
        self.name = schema["name"]

class Conversation:
    def __init__(self, system=None, tools=None, messages=None, filename=None):
        self.client = anthropic.Anthropic()
        self.model = "claude-3-5-sonnet-latest"
        self.system = system
        self.messages = messages or []
        self.tools = tools or []
        self.filename = filename
        if filename:
            try:
                with open(filename, "r") as file:
                    self.messages = json.load(file)
            except FileNotFoundError:
                print(f"File {filename} not found")
        
    def say(self, message):
        self.messages.append(
            {
                "role": "user", 
                "content": message
            }
        )
        system_message = self.system or self._generate_default_system_message()
        tools = [t.schema for t in self.tools]
        response = self.client.messages.create(
            model=self.model,
            system=system_message,
            messages=self.messages,
            max_tokens=3000,
            temperature=0.7,
            tools=tools,
        )

        # Handle potential tool use
        assistant_messages = []
        while response.stop_reason == "tool_use":
            print("\n"+str(response.content))

            tool_result_messages  = []
            for block in response.content:
                if block.type != "tool_use":
                    # TODO: this could be improved. As is we're going to just concatenate the text of all the blocks.
                    # This might not make sense to the reader since the tool calls are missing and the assistant might refer to them.
                    # An improvement would be to have the assistant say something like "I used the following tools to answer the question: ..."
                    # and then list the tools used and their results.
                    assistant_messages.append(block.text)
                else:
                    tool_use = block
                    tool_name = tool_use.name
                    tool_input = tool_use.input
                    tool_result = self._process_tool_call(tool_name, tool_input)
                    tool_result_messages.append({
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": tool_result,
                    })
            
            self.messages.append({"role": "assistant", "content": response.content})
            self.messages.append({
                "role": "user",
                "content": tool_result_messages,
            })
            
            # Get final response after tool use
            response = self.client.messages.create(
                model=self.model,
                system=system_message,
                messages=self.messages,
                max_tokens=3000,
                temperature=0.7,
                tools=tools,
            )
        
        assistant_messages.append(response.content[0].text)
        assistant_message = "\n".join(assistant_messages)
        self.messages.append({"role": "assistant", "content": assistant_message})
        if self.filename:
            self._save()
        print(assistant_message)
        return assistant_message
    
    def _process_tool_call(self, tool_name, tool_input):
        for tool in self.tools:
            if tool.name == tool_name:
                return tool.callable(tool_input)
        raise Exception(f"Tool {tool_name} not found")
    
    def _generate_default_system_message(self):
        return "You are a helpful assistant."

    def _save(self):
        if self.filename:
            with open(self.filename, "w") as file:
                messages_to_save = []
                for message in self.messages:
                    if isinstance(message['content'], list):
                        # Handle list of content blocks
                        new_content = []
                        for block in message['content']:
                            if hasattr(block, 'dict'):
                                new_content.append(block.dict())
                            else:
                                new_content.append(block)
                        messages_to_save.append({
                            'role': message['role'],
                            'content': new_content
                        })
                    else:
                        # Content is a simple string or already a dict
                        messages_to_save.append(message)
                json.dump(messages_to_save, file)


In [41]:
tools = [
    Tool(
        schema={
            "name": "get_stock_price",
            "description": "Retrieves the current stock price for a given ticker symbol. The ticker symbol must be a valid symbol for a publicly traded company on a major US stock exchange like NYSE or NASDAQ. The tool will return the latest trade price in USD. It should be used when the user asks about the current or most recent price of a specific stock. It will not provide any other information about the stock or company.",
            "input_schema": {
                "type": "object",
                "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g. AAPL for Apple Inc."
                }
                },
                "required": ["ticker"]
            }
        },
        callable=lambda x: "12.53",
    )
]
c = Conversation(tools=tools, filename="trash.json", system="You are a helpful assistant.")
c.say("What is the current stock price of Apple Inc and Microsoft?");

File trash.json not found

[TextBlock(text="I'll help you check the current stock prices for both Apple (AAPL) and Microsoft (MSFT) using the get_stock_price function.", type='text'), ToolUseBlock(id='toolu_013gkbJ4Xkqr5b32NdXPwwtu', input={'ticker': 'AAPL'}, name='get_stock_price', type='tool_use')]

[ToolUseBlock(id='toolu_01GarVA7jB8qkRUsD7rPtNXY', input={'ticker': 'MSFT'}, name='get_stock_price', type='tool_use')]
I'll help you check the current stock prices for both Apple (AAPL) and Microsoft (MSFT) using the get_stock_price function.
Here are the current stock prices:
- Apple (AAPL): $12.53
- Microsoft (MSFT): $12.53

These are the most recent trading prices for both companies in USD.


In [43]:
c = Conversation(tools=tools, filename="trash.json", system="You are a helpful assistant.")
c.say("try SDF")


[TextBlock(text="I'll check the current stock price for SDF.", type='text'), ToolUseBlock(id='toolu_01KZK8xwXSfrULMccqKvZ4b5', input={'ticker': 'SDF'}, name='get_stock_price', type='tool_use')]
I'll check the current stock price for SDF.
The current stock price for SDF is $12.53.


"I'll check the current stock price for SDF.\nThe current stock price for SDF is $12.53."